In [1]:
import pandas as pd
import sys
import os
import pyreadr

from cplvm import CPLVM
from cplvm import CPLVMLogNormalApprox

2024-05-01 16:19:50.881286: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [2]:
def return_indices_of_a(a, b):
  b_set = set(b)
  return [i for i, v in enumerate(a) if v in b_set]

In [3]:
ct = 'L2or3'
df = pyreadr.read_r('../pb/'+ ct + '.rds')[None]
hvg = pyreadr.read_r('../hvg1k/'+ ct +'.rds')[None]
input1 = df.index.values.tolist()
input2 = hvg[None].to_list()
match_id = return_indices_of_a(input1,input2)
# print(len(match_id))
df = df.iloc[match_id]
print(df.shape)

score_type = 'scoreA'
output_dir = './' + score_type + '_cplvm/'
if not os.path.exists(output_dir):
   os.makedirs(output_dir)

score = pyreadr.read_r('../'+ score_type + '_pfc.rds')[None]
print(score.shape)

(1000, 41)
(20, 5)


In [5]:
fg = df[score['sample'][score['diagnosis']=='ASD'].to_list()].to_numpy().T
print(type(fg))
print(fg.shape)

bg = df[score['sample'][score['diagnosis']=='Control'].to_list()].to_numpy().T
print(type(bg))
print(bg.shape)

<class 'numpy.ndarray'>
(10, 1000)
<class 'numpy.ndarray'>
(10, 1000)


In [6]:
X = bg
Y = fg

In [7]:
n, m = X.shape[0], Y.shape[0]
assert X.shape[1] == Y.shape[1]
p = X.shape[1]

In [8]:
# Set up CPLVM
cplvm = CPLVM(
    k_shared=8,
    k_foreground=8,
    compute_size_factors=True,
    offset_term=False)
approx_model = CPLVMLogNormalApprox(
    X.T, 
    Y.T, 
    k_shared=8, 
    k_foreground=8, 
    compute_size_factors=True, 
    offset_term=False
)
# Fit model
model_output = cplvm.fit_model_vi(
    X.T,
    Y.T,
    approximate_model=approx_model,
)

2024-05-01 16:24:01.390425: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2024-05-01 16:24:01.498482: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:82:00.0 name: NVIDIA GeForce GTX 1080 computeCapability: 6.1
coreClock: 1.7335GHz coreCount: 20 deviceMemorySize: 7.92GiB deviceMemoryBandwidth: 298.32GiB/s
2024-05-01 16:24:01.498536: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2024-05-01 16:24:01.562984: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2024-05-01 16:24:01.575795: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcufft.so.10
2024-05-01 16:24:01.584544: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuran

Instructions for updating:
`batch_shape` and `event_shape` args are deprecated. Please use `tfd.Sample`, `tfd.Independent`, and broadcasted parameters of the base distribution instead. For example, replace `tfd.TransformedDistribution(tfd.Normal(0., 1.), tfb.Exp(), batch_shape=[2, 3], event_shape=[4])` with `tfd.TransformedDistrbution(tfd.Sample(tfd.Normal(tf.zeros([2, 3]), 1.),sample_shape=[4]), tfb.Exp())` or `tfd.TransformedDistribution(tfd.Independent(tfd.Normal(tf.zeros([2, 3, 4]), 1.), reinterpreted_batch_ndims=1), tfb.Exp())`.
Instructions for updating:
`batch_shape` and `event_shape` args are deprecated. Please use `tfd.Sample`, `tfd.Independent`, and broadcasted parameters of the base distribution instead. For example, replace `tfd.TransformedDistribution(tfd.Normal(0., 1.), tfb.Exp(), batch_shape=[2, 3], event_shape=[4])` with `tfd.TransformedDistrbution(tfd.Sample(tfd.Normal(tf.zeros([2, 3]), 1.),sample_shape=[4]), tfb.Exp())` or `tfd.TransformedDistribution(tfd.Independent(

In [9]:
import numpy as np
# Foreground-specific loadings
W_mean = model_output["approximate_model"].qw_mean.numpy()
W_stddev = model_output["approximate_model"].qw_stddv.numpy()
W = np.exp(W_mean + W_stddev ** 2)
# Shared loadings
S_mean = model_output["approximate_model"].qs_mean.numpy()
S_stddev = model_output["approximate_model"].qs_stddv.numpy()
S = np.exp(S_mean + S_stddev ** 2)

In [20]:
print(S.shape)
print(W.shape)

(1000, 8)
(1000, 8)


In [21]:
W_df = pd.DataFrame(W, index=df.index.values)

In [22]:
W_df.sort_values(0,ascending=False).head(20)


,0,1,2,3,4,5,6,7
CX3CL1,2.653464,1.327651,1.676630,1.557988,1.308078,1.908956,2.051447,1.290851
MAPRE3,2.397135,1.494507,1.485439,1.517254,1.564504,1.797212,1.398085,1.614707
RAP1GAP2,2.392782,1.573007,1.511502,1.581340,1.502897,1.702924,1.561248,1.647786
RP11-34P1.2,2.300583,1.592253,1.548037,1.305574,1.609144,1.357154,1.841799,1.350908
MMD,2.276167,1.262197,1.852415,1.400500,1.819017,1.085254,1.525938,1.514041
HSP90AB1,2.263585,1.555006,1.746329,1.388367,2.122652,1.898005,1.576817,1.632223
ASIC2,2.249886,1.643525,1.365083,1.887834,2.171148,2.099802,2.003289,1.704126
COX4I1,2.239428,1.286585,1.377890,1.534498,1.426306,1.517490,1.916106,1.351248
MTND1P23,2.236998,1.256797,1.090050,1.540678,1.419309,1.282492,1.282597,1.533185
PHYHIP,2.231123,1.451944,1.553798,1.742764,1.536162,1.701686,1.893064,1.948942


In [23]:
W_df.sort_values(1,ascending=False).head(20)


,0,1,2,3,4,5,6,7
MALAT1,1.780805,2.489761,1.726937,1.656692,2.173221,2.161553,1.998178,2.285158
NAALADL2,1.497487,2.477368,1.610797,1.480138,1.553400,1.462173,1.623757,1.563922
UNC5D,2.063370,2.424596,1.658649,1.520033,2.317689,1.395861,1.658301,1.522318
CDH13,1.759789,2.344580,1.518392,1.751689,1.805680,1.785372,1.304020,1.956708
LINC01106,1.492755,2.337402,1.422817,1.775613,1.725707,1.400919,1.336429,1.391827
SNAP25-AS1,1.322246,2.322714,1.620678,1.591035,1.633878,1.724399,1.384565,1.620725
STMN2,1.745867,2.319617,1.960668,1.452794,1.464941,1.335819,1.815332,1.711709
MARCH1,1.800606,2.308681,1.615546,1.610761,1.462027,1.673818,1.623935,1.242659
SYN3,1.806001,2.287193,1.839763,1.702522,1.605367,1.519083,1.458760,1.438407
GRIK2,1.678939,2.258101,1.714132,2.088950,1.814923,1.776092,1.665146,1.630485


In [24]:
W_df.sort_values(2,ascending=False).head(20)


,0,1,2,3,4,5,6,7
ATP2B2,2.026523,1.783509,2.570445,1.520472,1.710451,1.334415,2.074463,1.555735
ATP1B1,1.919423,1.478754,2.269896,1.422086,1.460931,1.722351,1.857511,1.794925
GLS,1.770241,1.450061,2.264344,1.490567,1.700722,1.526923,1.372482,1.474779
SAT2,1.103781,1.087033,2.249904,1.270012,1.817208,1.574470,1.602731,1.364588
PCDH10,1.455790,1.952189,2.244375,1.370905,1.563040,1.566609,1.486985,1.171301
HOMER1,1.583028,1.347764,2.233500,1.522071,1.433464,1.603283,1.626516,1.691181
SPARCL1,1.923594,1.777803,2.228418,1.470919,1.478233,1.229402,1.837084,1.294879
CACNB2,1.705133,1.498754,2.207117,1.821471,1.577902,2.063138,1.289817,1.529252
SLC17A7,1.756371,1.847918,2.197932,1.812729,1.568154,1.244843,1.466325,1.269123
CNTNAP2,2.010451,1.917805,2.171269,1.585045,2.088392,1.662573,2.077141,1.962902


In [25]:
W_df.sort_values(3,ascending=False).head(20)


,0,1,2,3,4,5,6,7
HDAC9,1.599605,1.605599,1.638673,2.618804,1.372490,1.560445,1.762738,1.846970
RAPGEF2,1.776085,1.490743,1.268684,2.592069,1.894397,1.599397,1.576986,1.745983
RASGEF1B,1.403750,1.829620,1.865726,2.414375,1.739604,1.968361,2.003561,1.957690
EIF1,1.944776,1.445803,1.595713,2.311173,1.927960,1.640363,1.508618,1.309460
FSTL5,1.560560,1.437825,1.195411,2.292960,1.250100,1.147080,1.399799,1.720074
PLCL1,1.753458,1.812208,1.543933,2.274983,1.657665,1.247706,1.543290,1.399169
GRIN2B,2.150276,1.595821,1.918301,2.256571,2.309859,1.781506,1.630075,1.322344
MAST4,1.266933,1.662207,1.639922,2.240739,1.604130,1.698900,1.442040,1.745452
HS6ST3,1.502851,2.100291,1.888808,2.240037,1.723707,1.893501,2.038480,1.484450
OPCML,1.656404,1.705772,1.617259,2.216118,1.390827,1.270448,1.876099,1.870103


In [26]:
W_df.sort_values(4,ascending=False).head(20)


,0,1,2,3,4,5,6,7
LSAMP,1.730552,1.754023,1.994990,1.712955,2.575760,1.670599,2.124609,1.397589
RFX3,1.364575,2.167946,1.792816,1.418152,2.515548,1.647683,1.400272,1.483612
ARGLU1,1.399648,1.506166,1.282362,1.609465,2.397910,1.407568,1.412678,1.895691
STMN1,1.498302,1.948651,1.643420,1.669893,2.394702,1.580659,2.614221,1.741633
PLXDC2,1.680778,1.762254,1.317018,1.397930,2.388363,1.642045,1.243858,1.590595
CAMK1D,1.771849,1.833710,2.022188,1.648174,2.369039,2.075747,1.993265,1.328812
AP2S1,1.676699,1.444912,1.732638,1.632759,2.338852,1.218187,1.427532,1.411414
GPC1,1.144186,1.741173,1.542165,1.727001,2.336769,1.277204,1.291724,1.474883
LINC00685,1.214343,1.753526,1.492155,1.612147,2.320967,1.709338,1.347142,1.359325
UNC5D,2.063370,2.424596,1.658649,1.520033,2.317689,1.395861,1.658301,1.522318


In [27]:
W_df.to_csv(output_dir + "W_fg_loadings.csv")

In [28]:
S_df = pd.DataFrame(S, index=df.index.values)
S_df.to_csv(output_dir + "S_shared_loadings.csv")

In [33]:
t_mean = model_output["approximate_model"].qty_mean.numpy()
t_stddev = model_output["approximate_model"].qty_stddv.numpy()
t = np.exp(t_mean + t_stddev ** 2)

In [52]:
print(t.shape)
a=df[score['sample'][score['diagnosis']=='ASD'].to_list()]
t_df = pd.DataFrame(t, columns=a.columns)

(8, 10)


In [53]:
t_df

,5144_PFC,5278_PFC,5294_BA9,5403_PFC,5419_PFC,5565_BA9,5864_BA9,5939_BA9,5978_BA9,6033_BA9
0,0.000606,0.000244,0.000113,0.000046,0.000074,0.000092,0.015990,0.000132,0.000046,0.009154
1,0.000432,0.000147,0.000111,0.000043,0.000131,0.000236,0.042357,0.000072,0.000043,0.003704
2,0.000313,0.000260,0.000183,0.000047,0.000078,0.000082,0.019035,0.000150,0.000047,0.020363
3,0.000426,0.000306,0.000126,0.000047,0.000122,0.000171,0.002987,0.000162,0.000060,0.057281
4,0.000234,0.000251,0.000436,0.000035,0.000091,0.000076,0.019971,0.000087,0.000053,0.007873
5,0.000207,0.000899,0.000247,0.000037,0.000111,0.000099,0.003891,0.000065,0.000061,0.005562
6,0.000617,0.000204,0.000119,0.000038,0.000270,0.000146,0.029723,0.000063,0.000053,0.012201
7,0.000489,0.000219,0.000188,0.000040,0.000096,0.000298,0.019648,0.000152,0.000052,0.022923


In [54]:
t_df.to_csv(output_dir + "t_fg_factors.csv")